In [3]:
import numpy as np
import sys

sys.path.append("/Users/utilizador/Documents/GitHub/si/src")
from collections import Counter
from sklearn.tree import DecisionTreeClassifier
from si.data.dataset import Dataset
from si.io.csv_file import read_csv
from si.data.dataset import Dataset

from si.models.decision_tree_classifier import DecisionTreeClassifier
from collections import Counter


A classe RandomForestClassifier implementa o algoritmo Random Forest, um método de aprendizado de máquina supervisionado que combina múltiplas árvores de decisão para realizar classificações. Cada árvore é treinada em um subconjunto diferente dos dados com um subconjunto aleatório de características, criando um ensemble de modelos. A predição final é feita por votação majoritária entre as árvores.


__Random Forest__:
Um ensemble de Árvores de Decisão onde cada árvore é treinada com:
Um subconjunto aleatório das amostras (amostragem com reposição - bootstrap sampling).
Um subconjunto aleatório das características (feature bagging).
A diversidade entre as árvores reduz o risco de overfitting.

__Critérios de Impureza__-:
Gini: Mede a impureza das divisões em cada nó da árvore. Menor impureza indica melhor separação.
Entropia: Baseada na teoria da informação, mede a incerteza de uma divisão

__init__:
Inicializa o modelo com parâmetros configuráveis, como número de árvores e profundidade.

__fit__:
Cria n_estimators árvores de decisão, cada uma treinada em um subconjunto aleatório das amostras e características.

__predict__:
Faz predições para cada árvore no ensemble e combina os resultados por votação majoritária.

__score__:
Calcula a acurácia do modelo comparando as predições com os rótulos reais.

In [4]:

class RandomForestClassifier:
    """
    Random Forest Classifier: Combines multiple decision trees for classification
    """

    def __init__(self, n_estimators=1000, max_features=None, min_samples_split=2,
                 max_depth=10, mode='gini', seed=42):
        """
        Parameters:
        ----------
        n_estimators: int - Number of decision trees.
        max_features: int - Maximum features per tree.
        min_samples_split: int - Minimum samples to allow a split.
        max_depth: int - Maximum depth of the trees.
        mode: str - Criteria for splits ('gini' or 'entropy').
        seed: int - Random seed for reproducibility.
        """
        self.n_estimators = n_estimators
        self.max_features = max_features
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth  # Profundidade máxima das árvores
        self.mode = mode # Critério de impureza para as árvores
        self.seed = seed
        self.trees = []  

    def fit(self, dataset):
        """
        Train the random forest classifier.

        Parameters:
        ----------
        dataset: Dataset - Input data to fit the model.

        Returns:
        -------
        self: RandomForestClassifier - Trained model.
        """
        np.random.seed(self.seed)  # Ensures reproducibility
        n_samples, n_features = dataset.shape()
        
        if self.max_features is None:
            self.max_features = int(np.sqrt(n_features))

        for _ in range(self.n_estimators):
            sample_indices = np.random.choice(n_samples, n_samples, replace=True) # Seleciona amostras com reposição (bootstrap sampling)
            feature_indices = np.random.choice(n_features, self.max_features, replace=False)

            sampled_data = Dataset(dataset.X[sample_indices][:, feature_indices], dataset.y[sample_indices])

            # Inicia e treina a árvore
            tree = DecisionTreeClassifier(
                min_sample_split=self.min_samples_split,
                max_depth=self.max_depth,
                mode=self.mode
            )
            tree.fit(sampled_data)
            self.trees.append((feature_indices, tree))

        return self

    def predict(self, dataset):
        """
        Predict class labels for input dataset.

        Parameters:
        ----------
        dataset: Dataset - Input data for prediction.

        Returns:
        -------
        np.ndarray - Predicted class labels.
        """
        n_samples = dataset.shape()[0]
        predictions = np.zeros((self.n_estimators, n_samples), dtype=object)

        for i, (feature_indices, tree) in enumerate(self.trees):
            sampled_data = Dataset(dataset.X[:, feature_indices], dataset.y)
            predictions[i, :] = tree.predict(sampled_data)

        # Votação majoritária: Classe mais frequente para cada amostra
        majority_vote = np.apply_along_axis(lambda x: Counter(x).most_common(1)[0][0], axis=0, arr=predictions)
        return majority_vote

    def score(self, dataset):
        """
        Calculate model accuracy on dataset.

        Parameters:
        ----------
        dataset: Dataset - Input data for evaluation.

        Returns:
        -------
        float - Accuracy score.
        """
        predictions = self.predict(dataset)
        return np.mean(predictions == dataset.y)




In [5]:
Path= "/Users/utilizador/Documents/GitHub/si/datasets/iris/"
dataset = read_csv(Path + "iris.csv", sep=",", features= True, label= True)
dataset

In [6]:
from si.model_selection.split import stratified_train_test_split


train_data, test_data = stratified_train_test_split(dataset, test_size=0.3, random_state=42)


rf = RandomForestClassifier(n_estimators=1000, min_samples_split=3, max_features=2, max_depth=3, mode='gini', seed=42)

rf.fit(train_data)

test_accuracy = rf.score(test_data)

print(f"Model Accuracy on Test Set: {test_accuracy}")

Model Accuracy on Test Set: 0.3333333333333333
